In [ ]:
# Google Colab Setup / Google Colab セットアップ
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import sys
    from pathlib import Path
    
    proj_root = Path("/content/drive/MyDrive/image_processing_tutorial-main")
    if str(proj_root) not in sys.path:
        sys.path.insert(0, str(proj_root))
    
    print("Google Colab environment detected and configured!")
    print("Google Colab環境が検出され、設定されました！")
else:
    print("Local Jupyter environment detected")
    print("ローカルJupyter環境が検出されました")

# Machine Learning Basics: Linear Regression and Model Selection

This notebook demonstrates fundamental machine learning concepts using linear regression with f=ma relationship data. We will explore model selection using cross-validation.

このノートブックでは、f=ma関係のデータを使用した線形回帰により、機械学習の基本概念を実演します。交差検証を使用したモデル選択を探求します。

- Generate artificial data based on physical relationships / 物理的関係に基づく人工データの生成
- Implement linear regression with and without intercept / 切片ありなしの線形回帰の実装
- Understand cross-validation for model evaluation / モデル評価のための交差検証の理解
- Compare model performance using statistical metrics / 統計的指標を使用したモデル性能の比較

In [ ]:
# Import required libraries / 必要なライブラリをインポート
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model  # sklearn.linear_model.LinearRegression
from sklearn.model_selection import LeaveOneOut  # cross validation
from sklearn.metrics import mean_squared_error  # RMSE

np.random.seed(42)

## 1. Artificial Data Generation / 人工データ生成

We will generate artificial data following a linear relationship similar to Newton's second law (F = ma).
ニュートンの第二法則（F = ma）に類似した線形関係に従って人工データを生成します。

In [ ]:
# Artificial data generation

N = 21  # # of samples
a = 1
X = np.linspace(-10, 10, N)  # -10, 10 to N equally divided data.

noise_cov = 3
noise = np.random.normal(0, noise_cov, [N])
y = a * X + noise

print(f"Generated {N} data points with linear relationship y = {a} * X + noise")
print(f"線形関係 y = {a} * X + ノイズ で {N} 個のデータポイントを生成")

### Q1: Plot the artificial data / 問１：人工データのプロットしてください

Plot the generated data points to visualize the linear relationship with noise.
ノイズを含む線形関係を可視化するために生成されたデータポイントをプロットしてください。

In [ ]:
# Q1: Plot the artificial data.
plt.plot(X, y, 'r.')
plt.axis('equal')
plt.title('DATA')
plt.xlabel('X')
plt.ylabel('y')
plt.show()

## 2. Linear Regression Model Setup / 線形回帰モデルの設定

We will create two different linear models:
1. Model with intercept: y = ax + b
2. Model without intercept: y = ax

2つの異なる線形モデルを作成します：
1. 切片ありモデル：y = ax + b  
2. 切片なしモデル：y = ax

### Q2: Generate linear models with and without intercept / 問2: １次式のモデルのうち，切片があるモデルとないモデルをそれぞれ生成してください

Create two LinearRegression models using scikit-learn with different intercept settings.
異なる切片設定でscikit-learnを使用して2つのLinearRegressionモデルを作成してください。

In [ ]:
# Q2: Generate a linear model with and without intercept, respectively.
reg_ax_b = linear_model.LinearRegression(fit_intercept=True)  ## (option to generate a model with an intercept)
reg_ax = linear_model.LinearRegression(fit_intercept=False)   ## (option to generate a model with no intercept)

print("Created two models:")
print("2つのモデルを作成:")
print("- reg_ax_b: Linear regression with intercept (切片あり)")
print("- reg_ax: Linear regression without intercept (切片なし)")

## 3. Cross-Validation Setup / 交差検証の設定

We will use Leave-One-Out cross-validation to evaluate model performance.
Leave-One-Out交差検証を使用してモデル性能を評価します。

In [ ]:
# Evaluate performance with K-partition cross validation

loo = LeaveOneOut()  # Generating training and test data set for leave one out cross validation
MSE_ax = {"mean": [], "std": []}
MSE_ax_b = {"mean": [], "std": []}

scores_ax = []
scores_ax_b = []

print("Leave-One-Out cross-validation setup complete")
print("Leave-One-Out交差検証の設定完了")

### Q3: Print train_index and test_index / 問3: printをもちいて，train_index, test_indexをプリントし、テストデータとトレーニングデータを確認してください

Examine the cross-validation splits to understand how Leave-One-Out works.
Leave-One-Outの動作を理解するために交差検証の分割を調べてください。

In [ ]:
# Q3: Print train_index and test_index using print and check the test and training data.

for train_index, test_index in loo.split(X):
    print("%s %s" % (train_index, test_index))

### Q4 & Q5: Model Training and Prediction / 問４・５：モデル学習と予測

**Q4**: Train linear regression models using training data / 線形回帰のモデルをトレーニングデータを用いて学習してください

**Q5**: Make predictions on test data using trained models / 学習済みモデルを用いてテストデータについて予測してください

In [ ]:
# main part: leave one out cross validation
for train_index, test_index in loo.split(X):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    reg_ax_b.fit(X_train[:, np.newaxis], y_train[:, np.newaxis])
    reg_ax.fit(X_train[:, np.newaxis], y_train[:, np.newaxis])
    
    pred_ax_b = reg_ax_b.predict(X_test[:, np.newaxis])
    pred_ax = reg_ax.predict(X_test[:, np.newaxis])
    
    scores_ax_b.append(mean_squared_error(y_test, pred_ax_b))
    scores_ax.append(mean_squared_error(y_test, pred_ax))

MSE_ax_b["mean"].append(np.mean(scores_ax_b))  # convert to positive
MSE_ax["mean"].append(np.mean(scores_ax))

print('Cross-validation error when y= ax+b as model: %.4f' % MSE_ax_b["mean"][0])
print('Cross-validation error when y= ax as model: %.4f' % MSE_ax["mean"][0])
print('y= ax+bをモデルとしたときの交差検証誤差： %.4f' % MSE_ax_b["mean"][0])
print('y= axをモデルとしたときの交差検証誤差： %.4f' % MSE_ax["mean"][0])

## 4. Results Analysis / 結果分析

Compare the cross-validation errors between the two models to determine which performs better.
2つのモデル間の交差検証誤差を比較して、どちらが優れた性能を示すかを判定します。

- **Model 1 (y = ax)**: Linear regression without intercept / 切片なし線形回帰
- **Model 2 (y = ax + b)**: Linear regression with intercept / 切片あり線形回帰

The model with lower cross-validation error is considered better for this dataset.
交差検証誤差が低いモデルがこのデータセットに対してより良いと考えられます。

## 5. Conclusion / 結論

Based on the cross-validation results, we can make an informed decision about model selection:

交差検証結果に基づいて、モデル選択について情報に基づいた決定を行うことができます：

1. **Data Generation**: Created artificial data with known linear relationship / データ生成：既知の線形関係を持つ人工データを作成
2. **Model Comparison**: Compared models with and without intercept / モデル比較：切片ありなしのモデルを比較
3. **Cross-Validation**: Used Leave-One-Out CV for robust evaluation / 交差検証：堅牢な評価のためにLeave-One-Out CVを使用
4. **Model Selection**: Selected best model based on CV error / モデル選択：CV誤差に基づいて最適モデルを選択

This demonstrates the fundamental machine learning workflow of data generation, model training, evaluation, and selection.
これは、データ生成、モデル学習、評価、選択という機械学習の基本的なワークフローを実演しています。